This file implements a time-series forecasting model using Prophet to predict the number of patient visits. The project includes forecasting for both inpatient and outpatient services, as well as a performance evaluation for each model.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from prophet import Prophet
from prophet.make_holidays import make_holidays_df
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import itertools

Data Preparation

In [ ]:
# Inpatient Data: Preparation for Prophet model
df_daily_inpatient = df_rawatinap.groupby('tanggal_admisi').agg(
    visit_count=('rawat_inap_id', 'count')
).reset_index().rename(columns={'tanggal_admisi': 'ds', 'visit_count': 'y'})

# Outpatient Data: Preparation for Prophet model
df_daily_outpatient = df_rawatjalan.groupby('tanggal_kunjungan').agg(
    visit_count=('rawat_jalan_id', 'count')
).reset_index().rename(columns={'tanggal_kunjungan': 'ds', 'visit_count': 'y'})

# Add national holidays for both datasets
years_inap = df_daily_inpatient['ds'].dt.year.unique().tolist()
holidays_inap = make_holidays_df(year_list=years_inap, country='ID')

years_jalan = df_daily_outpatient['ds'].dt.year.unique().tolist()
holidays_jalan = make_holidays_df(year_list=years_jalan, country='ID')

Forecasting Model Implementation (Inpatient)

In [ ]:
# Implement Prophet model for inpatient data
model_inap = Prophet(holidays=holidays_inap)
model_inap.fit(df_daily_inpatient)

# Create future dataframe for prediction
future_inap = model_inap.make_future_dataframe(periods=180)
forecast_inap = model_inap.predict(future_inap)

# --- Hyperparameter Optimization ---
best_params_inap = {
    'seasonality_mode': 'additive',
    'changepoint_prior_scale': 0.5
}
best_model_inap = Prophet(holidays=holidays_inap, **best_params_inap)
best_model_inap.fit(df_daily_inpatient)
forecast_best_inap = best_model_inap.predict(future_inap)

# --- Model Evaluation ---
# Use MAE, RMSE, and R² to measure accuracy
df_eval_filtered_inap = df_daily_inpatient.copy()
df_eval_filtered_inap = df_eval_filtered_inap.merge(
    forecast_best_inap[['ds', 'yhat']], on='ds', how='left'
).dropna()

mae_inap = mean_absolute_error(df_eval_filtered_inap['y'], df_eval_filtered_inap['yhat'])
rmse_inap = mean_squared_error(df_eval_filtered_inap['y'], df_eval_filtered_inap['yhat'], squared=False)
r2_inap = r2_score(df_eval_filtered_inap['y'], df_eval_filtered_inap['yhat'])
mean_actual_inap = df_eval_filtered_inap['y'].mean()
mae_pct_inap = mae_inap / mean_actual_inap * 100
rmse_pct_inap = rmse_inap / mean_actual_inap * 100

print("=== Inpatient Evaluation ===")
print(f"MAE  : {mae_inap:.2f} ({mae_pct_inap:.2f}%)")
print(f"RMSE : {rmse_inap:.2f} ({rmse_pct_inap:.2f}%)")
print(f"R²   : {r2_inap:.4f}")

Forecasting Model Implementation (Outpatient)

In [ ]:
# Implement Prophet model for outpatient data
model_jalan = Prophet(holidays=holidays_jalan)
model_jalan.fit(df_daily_outpatient)

# Create future dataframe for prediction
future_jalan = model_jalan.make_future_dataframe(periods=360)
forecast_jalan = model_jalan.predict(future_jalan)

# --- Hyperparameter Optimization ---
# Assumption: Best parameters have been found from the tuning process.
best_params_jalan = {
    'seasonality_mode': 'multiplicative',
    'changepoint_prior_scale': 0.1
}
best_model_jalan = Prophet(holidays=holidays_jalan, **best_params_jalan)
best_model_jalan.fit(df_daily_outpatient)
forecast_best_jalan = best_model_jalan.predict(future_jalan)

# --- Model Evaluation ---
# Use MAE, RMSE, and R² to measure accuracy
df_eval_filtered_jalan = df_daily_outpatient.copy()
df_eval_filtered_jalan = df_eval_filtered_jalan.merge(
    forecast_best_jalan[['ds', 'yhat']], on='ds', how='left'
).dropna()

mae_jalan = mean_absolute_error(df_eval_filtered_jalan['y'], df_eval_filtered_jalan['yhat'])
rmse_jalan = mean_squared_error(df_eval_filtered_jalan['y'], df_eval_filtered_jalan['yhat'], squared=False)
r2_jalan = r2_score(df_eval_filtered_jalan['y'], df_eval_filtered_jalan['yhat'])
mean_actual_jalan = df_eval_filtered_jalan['y'].mean()
mae_pct_jalan = mae_jalan / mean_actual_jalan * 100
rmse_pct_jalan = rmse_jalan / mean_actual_jalan * 100

print("\n=== Outpatient Evaluation ===")
print(f"MAE  : {mae_jalan:.2f} ({mae_pct_jalan:.2f}%)")
print(f"RMSE : {rmse_jalan:.2f} ({rmse_pct_jalan:.2f}%)")
print(f"R²   : {r2_jalan:.4f}")

Visualization of Result

In [ ]:
# Visualize inpatient forecast results
fig_inap = best_model_inap.plot(forecast_best_inap)
plt.title("Inpatient Visit Forecast (Historical + Predicted)")
plt.xlabel("Month")
plt.ylabel("Number of Patients")
plt.show()

# Visualize outpatient forecast results
fig_jalan = best_model_jalan.plot(forecast_best_jalan)
plt.title("Outpatient Visit Forecast (Historical + Predicted)")
plt.xlabel("Month")
plt.ylabel("Number of Visits")
plt.show()